# Setup

In [ ]:
# Install Tensorboard
!pip install tensorboardX --quiet
!pip install tensorboard --quiet


In [ ]:
# Install deps
! pip install fairseq sacrebleu sentencepiece --quiet


In [ ]:
# Install nvidia apex for multi-tensor support
# Ref: https://zhuanlan.zhihu.com/p/194176917
%cd /home/ubuntu/nmt
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

import fairseq
print(fairseq.utils.multi_tensor_l2norm_available)


## Downloads

In [ ]:
# FLoRes repo clone
%cd /home/ubuntu/nmt
!git clone https://github.com/facebookresearch/flores.git


In [ ]:
# get mBART pretrained model
%cd /home/ubuntu/nmt/flores/
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
!tar -xzvf mbart.cc25.v2.tar.gz
!rm -rf mbart.cc25.v2.tar.gz


# CodeEnv Setup

In [ ]:
SRC="en"
TGT="km"
BPE=1
NUM_UPDATES=45000
SAVE_INTERVAL=15000

In [ ]:
%cd /home/ubuntu/nmt/
!cp -f download_data.sh ./flores/download_data.sh
!cp -f prepare_parallel.sh ./flores/prepare_parallel.sh
!cp -f prepare_reverse.sh ./flores/prepare_reverse.sh


In [ ]:
# get baseline model
!mkdir -p /home/ubuntu/nmt/flores/checkpoints
%cd /home/ubuntu/nmt/flores/checkpoints
!wget https://msainmtntu.s3-ap-southeast-1.amazonaws.com/checkpoints/"checkpoint_best_parallel_"$TGT"_"$SRC".pt"
# !mv "checkpoint_best_parallel_"$TGT"_"$SRC".pt" checkpoint_best.pt

# Data Setup

In [ ]:
# Training data
%cd /home/ubuntu/nmt/flores/
!bash download-data.sh --langs $TGT"-"$SRC

In [ ]:
%cd /home/ubuntu/nmt/flores/
! bash prepare_reverse.sh --src $SRC --tgt $TGT --bpe $BPE

# Fairseq-train

In [ ]:
#PRETRAIN=mbart.cc25.v2/model.pt # fix if you moved the downloaded checkpoint
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/"wiki_"$SRC"_"$TGT"_bpe5000/" \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang $SRC --target-lang $TGT \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update $NUM_UPDATES \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_parallel_"$TGT"_"$SRC".pt" \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir "Tensorboard"$SRC$TGT/ \
  --max-update $NUM_UPDATES \
  --checkpoint-suffix "_reverse_"$SRC"_"$TGT \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a "mbart-moses-"$SRC"-"$TGT"-screenlog"


In [ ]:
#########################################################################################################
# Reverse training for BL
#########################################################################################################
# EN-SI
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_en_si_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang en --target-lang si \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 45000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_si_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir Tensorboard/ \
  --max-update 45000 \
  --checkpoint-suffix "_reverse_en_si" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-en-si-screenlog

#########################################################################################################
# EN-NE
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_en_ne_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang en --target-lang ne \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 45000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_ne_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir Tensorboard/ \
  --max-update 45000 \
  --checkpoint-suffix "_reverse_en_ne" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-en-ne-screenlog

#########################################################################################################
# EN-PS
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_en_ps_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang en --target-lang ps \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 45000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_ps_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir Tensorboard/ \
  --max-update 45000 \
  --checkpoint-suffix "_reverse_en_ps" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-en-ps-screenlog

#########################################################################################################
# EN-KM
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_en_km_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang en --target-lang km \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 45000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parallel_km_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir Tensorboard/ \
  --max-update 45000 \
  --checkpoint-suffix "_reverse_en_km" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-en-km-screenlog
  

# Eval

In [ ]:
#Generate Sacrebleu
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
!fairseq-generate ./data-bin/"wiki_"$SRC"_"$TGT"_bpe5000/" \
    --source-lang $SRC --target-lang $TGT \
    --path /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_reverse_"$SRC"_"$TGT".pt" \
    --task translation_from_pretrained_bart \
    --lenpen 1.2 \
    --gen-subset test \
    --bpe 'sentencepiece' --sentencepiece-model mbart.cc25.v2/sentence.bpe.model \
    --scoring sacrebleu  --batch-size 32 --langs $langs > $SRC"_"$TGT"_bleu_w_bpe"

!tail -1 $SRC"_"$TGT"_bleu_w_bpe"


In [ ]:
# !cat $SRC"_"$TGT"_bleu_w_bpe" | grep -P "^D" |sort -V |cut -f 3- | sed 's/\[en_XX\]//g' > $SRC"_"$TGT".hyp"
# !cat $SRC"_"$TGT"_bleu_w_bpe" | grep -P "^T" |sort -V |cut -f 2- | sed 's/\[en_XX\]//g' > $SRC"_"$TGT".ref"
# import sacrebleu
# my_hyp_file = open($SRC"_"$TGT".hyp", "r")
# hyp = my_hyp_file.readlines()
# my_ref_file = open($SRC"_"$TGT".ref", "r")
# ref_tmp = my_ref_file.readlines()
# ref = [sent for sent in ref_tmp]
# ref =[ref]
# bleu = sacrebleu.corpus_bleu(hyp,ref)
# print(bleu.score)

# Housekeeping

In [ ]:
# Upload checkpoint
!aws s3 cp /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_reverse_"$SRC"_"$TGT".pt" s3://msainmtntu/checkpoints/"checkpoint_best_reverse_"$SRC"_"$TGT".pt"
!rm -rf /home/ubuntu/nmt/flores/checkpoints/*"0000_reverse_"$SRC"_"$TGT".pt"


In [ ]:
# Upload tensorboard
!aws s3 cp --recursive /home/ubuntu/nmt/flores/Tensorboard s3://msainmtntu/Tensorboards/TensorboardReverseBaseline$(echo "$SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z)


## Checks

In [ ]:
!ls -lh /home/ubuntu/nmt/flores/checkpoints/

In [ ]:
!tail -10 /home/ubuntu/nmt/flores/"mbart-moses-"$SRC"-"$TGT"-screenlog"